# Аналитика в сервисе для чтения книг по подписке

## Оглавление

1. [Описание проекта](#projekt) 
2. [Цель и задачи проекта](#purpose)     
3. [Описание данных](#data)    
4. [Выгрузка данных и изучение общей информации](#start)
5. [Исследование данных](#analysis)   
6. [Общий вывод](#conclusion)

## Описание проекта
<a name="projekt"></a>

Компания купила крупный сервис для чтения книг по подписке. Первая задача - проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

## Цель и задачи проекта
<a name="purpose"></a>

Цель проекта - формулировка ценностного предложения для нового продукта.

<br>Для этого необходимо:     
1. посчитать, сколько книг вышло после 1 января 2000 года;   
2. для каждой книги посчитать количество обзоров и среднюю оценку;   
3. определить издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры);   
4. определить автора с самой высокой средней оценкой книг (учесть только книги с 50 и более оценками);   
5. посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных
<a name="data"></a>

#### Таблица ` books` (cодержит данные о книгах)

-  book_id - идентификатор книги   
-  author_id - идентификатор автора  
-  title - название книги   
-  num_pages - количество страниц   
-  publication_date - дата публикации книги  
-  publisher_id - идентификатор издателя  

#### Таблица ` authors` (cодержит данные об авторах)

-  author_id - идентификатор автора   
-  author - имя автора

#### Таблица ` publishers` (cодержит данные об издательствах)

-  publisher_id - идентификатор издательства   
-  publisher - название издательства

#### Таблица ` ratings` (cодержит данные о пользовательских оценках книг)

-  rating_id - идентификатор оценки   
-  book_id - идентификатор книги   
-  username - имя пользователя, оставившего оценку   
-  rating - оценка книги

#### Таблица ` reviews` (cодержит данные о пользовательских обзорах)

-  review_id - идентификатор обзора  
-  book_id - идентификатор книги  
-  username - имя автора обзора  
-  text - текст обзора

#### Схема таблиц в базе данных: 
![relationship_of_the_tables](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)   

## Выгрузка данных и изучение общей информации
<a name="start"></a>

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# %%HTML
# <style type="text/css">
# table.dataframe td, table.dataframe th {
#     border: 1px  black solid !important;
#   color: black !important;
# }

In [3]:
# создадим коннекцию к базе
db_config = {'user' : 'praktikum_student', # имя пользователя
             'pwd' : 'Sdf4$2;d-d30pp', # пароль
             'host' : 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port' : 6432, # порт подключения
             'db' : 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
engine = create_engine(connection_string, connect_args = {'sslmode':'require'}) 

In [4]:
# создадим функцию для вывода результатов запроса
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine)

#### Таблица ` books` (cодержит данные о книгах)

In [5]:
# посмотрим на 5 верхних строк таблицы
sql = ''' 
            SELECT *
            FROM books
            LIMIT 5
      '''
select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
# узнаем размер таблицы
sql = ''' 
            SELECT COUNT(book_id) length
            FROM books
      '''
select(sql)

,length
0,1000


Таблица состоит из 1000 строк, 6 столбцов.

In [7]:
# изучим общую информацию таблицы 
sql = ''' 
            SELECT COUNT (DISTINCT book_id) books_count,
                   COUNT (DISTINCT author_id) authors_count,
                   COUNT (DISTINCT title) titles_count,
                   MIN (num_pages) min_pages,
                   MAX (num_pages) max_pages,
                   CAST(AVG(num_pages) AS numeric(4, 1)) avg_pages,
                   MIN (publication_date) min_publication_date,
                   MAX (publication_date) max_publication_date,
                   COUNT (DISTINCT publisher_id) publishers_count
            FROM books
      '''
select(sql)

,books_count,authors_count,titles_count,min_pages,max_pages,avg_pages,min_publication_date,max_publication_date,publishers_count
0,1000,636,999,14,2690,389.1,1952-12-01,2020-03-31,340


В таблице имеется информация о 1000 уникальных книгах, написанных 636 авторами. У двух книг имеются одинаковые названия. Минимальное количество страниц в книге - 14, максимальное - 2690, среднее - 389,1. Книги были опубликованы в 340 издательствах в период с 1 декабря 1952 года по 31 марта 2020 года.

Проверим, что строки с книгами с одинаковыми названиями не являются дубликатами.

In [8]:
# посмотрим на книги с одинаковыми названиями
sql = ''' 
            SELECT *
            FROM books
            WHERE title IN (
                            SELECT title
                            FROM books
                            GROUP BY title
                            HAVING COUNT (title) > 1
                            )
      '''
select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Книги с одинаковым названием написаны одним и тем же автором, однако опубликованы в разных изданиях и имеют разное количество страниц, не будем считать эти строки дубликатами.

Проверим, нет ли в таблице пропущенных значений.

In [9]:
# проверим таблицу на наличие пропущенных значений
sql = ''' 
            SELECT *
            FROM books
            WHERE (author_id IS NULL) OR
                  (num_pages IS NULL) OR
                  (publication_date IS NULL) OR
                  (publisher_id IS NULL)
      '''
select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id


Пропущенные значения в таблице отсутствуют.

#### Таблица ` authors` (cодержит данные об авторах)

In [10]:
# посмотрим на 5 верхних строк таблицы
sql = ''' 
            SELECT *
            FROM authors
            LIMIT 5
      '''
select(sql)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [11]:
# узнаем размер таблицы
sql = ''' 
            SELECT COUNT(author_id) length
            FROM authors
      '''
select(sql)

,length
0,636


Таблица состоит из 636 строк, 2 столбцов.

In [12]:
# изучим общую информацию таблицы 
sql = ''' 
            SELECT COUNT (DISTINCT author_id) authors_count,
                   COUNT (DISTINCT author) authors_name_count
            FROM authors
      '''
select(sql)

,authors_count,authors_name_count
0,636,636


В таблице имеется информация о 636 уникальных авторах. Дубликаты и пропущенные значения отсутствуют.

#### Таблица ` publishers` (cодержит данные об издательствах)

In [13]:
# посмотрим на 5 верхних строк таблицы
sql = ''' 
            SELECT *
            FROM publishers
            LIMIT 5
      '''
select(sql)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [14]:
# узнаем размер таблицы
sql = ''' 
            SELECT COUNT(publisher_id) length
            FROM publishers
      '''
select(sql)

,length
0,340


Таблица состоит из 340 строк, 2 столбцов.

In [15]:
# изучим общую информацию таблицы 
sql = ''' 
            SELECT COUNT (DISTINCT publisher_id) publishers_count,
                   COUNT (DISTINCT publisher) publishers_name_count
            FROM publishers
      '''
select(sql)

,publishers_count,publishers_name_count
0,340,340


В таблице имеется информация о 340 уникальных издательствах. Дубликаты и пропущенные значения отсутствуют.

#### Таблица ` ratings` (cодержит данные о пользовательских оценках книг)

In [16]:
# посмотрим на 5 верхних строк таблицы
sql = ''' 
            SELECT *
            FROM ratings
            LIMIT 5
      '''
select(sql)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [17]:
# узнаем размер таблицы
sql = ''' 
            SELECT COUNT(rating_id) length
            FROM ratings
      '''
select(sql)

,length
0,6456


Таблица состоит из 6456 строк, 4 столбцов.

In [18]:
# изучим общую информацию таблицы 
sql = ''' 
            SELECT COUNT (DISTINCT rating_id) ratings_count,
                   COUNT (DISTINCT book_id) books_count,
                   COUNT (DISTINCT username) usernames_count,
                   MIN (rating) min_rating,
                   MAX (rating) max_rating,
                   CAST(AVG(rating) AS numeric(3, 2)) avg_rating
            FROM ratings
      '''
select(sql)

,ratings_count,books_count,usernames_count,min_rating,max_rating,avg_rating
0,6456,1000,160,1,5,3.93


В таблице имеется информация о 6456 пользовательских оценках книг, поставленных 160 читателями. Оценки читателями были выставлены всем 1000 книгам. Минимальная пользовательская оценка - 1, максимальная - 5, средняя - 3,93. 

Проверим, нет ли в таблице пропущенных значений.

In [19]:
# проверим таблицу на наличие пропущенных значений
sql = ''' 
            SELECT *
            FROM ratings
            WHERE (book_id IS NULL) OR
                  (username IS NULL) OR
                  (rating IS NULL)
      '''
select(sql)

,rating_id,book_id,username,rating


Пропущенные значения в таблице отсутствуют.

#### Таблица ` reviews` (cодержит данные о пользовательских обзорах)

In [20]:
# посмотрим на 5 верхних строк таблицы
sql = ''' 
            SELECT *
            FROM reviews
            LIMIT 5
      '''
select(sql)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [21]:
# узнаем размер таблицы
sql = ''' 
            SELECT COUNT(review_id) length
            FROM reviews
      '''
select(sql)

,length
0,2793


Таблица состоит из 2793 строк, 4 столбцов.

In [22]:
# изучим общую информацию таблицы 
sql = ''' 
            SELECT COUNT (DISTINCT review_id) reviews_count,
                   COUNT (DISTINCT book_id) books_count,
                   COUNT (DISTINCT username) usernames_count,
                   COUNT (DISTINCT text) texts_count
            FROM reviews
      '''
select(sql)

,reviews_count,books_count,usernames_count,texts_count
0,2793,994,160,2793


В таблице имеется информация о 2793 обзорах книг, подготовленных 160 пользователями. Обзоры читателями были подготовлены по всем книгам, за исключением 6. Текст всех обзоров является уникальным. 

Проверим, нет ли в таблице пропущенных значений.

In [23]:
# проверим таблицу на наличие пропущенных значений
sql = ''' 
            SELECT *
            FROM reviews
            WHERE (book_id IS NULL) OR
                  (username IS NULL) OR
                  (text IS NULL)
      '''
select(sql)

,review_id,book_id,username,text


Пропущенные значения в таблице отсутствуют.

### Вывод

При изучении таблиц с данными установлено:   

Таблица ` books` (cодержит данные о книгах):  
  -  таблица состоит из 1000 строк, 6 столбцов, пропущенные значения и дубликаты отсутствуют;  
  -  в таблице имеется информация о 1000 уникальных книгах, написанных 636 авторами;  
  -  у двух книг имеются одинаковые названия, однако они хоть и написаны одним и тем же автором, опубликованы в разных изданиях и имеют разное количество страниц;  
  -  минимальное количество страниц в книге - 14, максимальное - 2690, среднее - 389,1;   
  -  книги были опубликованы в 340 издательствах в период с 1 декабря 1952 года по 31 марта 2020 года.      

Таблица ` authors` (cодержит данные об авторах):  
  -  таблица состоит из 636 строк, 2 столбцов, пропущенные значения и дубликаты отсутствуют;  
  -  в таблице имеется информация о 636 уникальных авторах.   

Таблица ` publishers` (cодержит данные об издательствах):  
  -  таблица состоит из 340 строк, 2 столбцов, пропущенные значения и дубликаты отсутствуют;  
  -  в таблице имеется информация о 340 уникальных издательствах.   

Таблица ` ratings` (cодержит данные о пользовательских оценках книг):  
  -  таблица состоит из 6456 строк, 4 столбцов, пропущенные значения и дубликаты отсутствуют;  
  -  в таблице имеется информация о 6456 пользовательских оценках книг, поставленных 160 читателями;  
  -  оценки читателями были выставлены всем 1000 книгам;  
  -  минимальная пользовательская оценка - 1, максимальная - 5, средняя - 3,93.   

Таблица ` reviews` (cодержит данные о пользовательских обзорах):  
  -  таблица состоит из 2793 строк, 4 столбцов, пропущенные значения и дубликаты отсутствуют;  
  -  в таблице имеется информация о 2793 обзорах книг, подготовленных 160 пользователями;  
  -  обзоры читателями были подготовлены по всем книгам, за исключением 6;  
  -  текст всех обзоров является уникальным.  

## Исследование данных
<a name="analysis"></a>

#### Сколько книг вышло после 1 января 2000 года?

In [24]:
# выполним sql-запрос
sql = ''' 
            SELECT COUNT(book_id) books_count
            FROM books
            WHERE publication_date >= '2000-01-01'
      '''
select(sql)

,books_count
0,821


Из 1000 книг, имеющихся в базе данных, после 1 января 2000 года была опубликована 821.

#### Количество обзоров и средняя оценка для каждой книги.

In [25]:
# выполним sql-запрос с сортировкой по количеству обзоров
sql = ''' 
            SELECT title,
                   author,
                   COUNT(DISTINCT review_id) reviews_count,
                   cast(AVG(rating) AS numeric(3, 2)) avg_rating
            FROM ratings rt
            LEFT JOIN reviews rw ON rw.book_id = rt.book_id
            LEFT JOIN books b ON b.book_id = rt.book_id
            LEFT JOIN authors a ON a.author_id = b.author_id
            GROUP BY rt.book_id,
                     title,
                     author
            ORDER BY reviews_count DESC,
                     avg_rating DESC
      '''
select(sql)

,title,author,reviews_count,avg_rating
0,Twilight (Twilight #1),Stephenie Meyer,7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,6,4.41
2,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré,6,4.29
3,The Book Thief,Markus Zusak/Cao Xuân Việt Khương,6,4.26
4,The Glass Castle,Jeannette Walls,6,4.21
...,...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,Teddy Slater/Ron Dias/Ric González,0,4.00
996,Leonardo's Notebooks,Leonardo da Vinci/H. Anna Suh,0,4.00
997,Essential Tales and Poems,Edgar Allan Poe/Benjamin F. Fisher,0,4.00
998,Anne Rice's The Vampire Lestat: A Graphic Novel,Faye Perozich/Anne Rice/John Bolton/Daerick Gröss,0,3.67


Максимальное количество обзоров пользователей у книги "Сумерки" писательницы Стефани Майер - 7, однако ее средняя пользовательская оценка достаточно невысока - всего 3,66. У 6 книг обзоры отсутствуют.

Максимальная средняя пользовательская оценка - 5,0 - у достаточно большого количества книг, при этом максимальное количество обзоров у них - 4. Минимальная средняя пользовательская оценка - 1,5 - у одной книги - "Забрать любовь" писательницы Джоди Пиколт.

In [27]:
# выполним sql-запрос с подсчетом количества книг со средней пользовательской оценкой 5
sql = ''' 
            SELECT COUNT(book_id)
            FROM (
                  SELECT book_id,
                         cast(AVG(rating) AS numeric(3, 2)) avg_rating
                  FROM ratings
                  GROUP BY book_id
                  ) dt
            WHERE avg_rating = 5      
      '''
select(sql)

,count
0,44


Максимальная средняя пользовательская оценка - 5,0 - у 44 книг.

#### Издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры).

In [28]:
# выполним sql-запрос
sql = ''' 
            SELECT publisher,
                   COUNT(book_id) books_count
            FROM books b
            LEFT JOIN publishers p ON p.publisher_id = b.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY books_count DESC
            LIMIT 1
      '''
select(sql)

,publisher,books_count
0,Penguin Books,42


Максимальное количество книг толще 50 страниц - 42 - выпустило издательство "Penguin Books".

#### Автор с самой высокой средней оценкой книг (учесть только книги с 50 и более оценками).

In [29]:
# выполним sql-запрос
sql = ''' 
            SELECT author,
                   cast(AVG(rating) AS numeric(3, 2)) AS avg_rating
            FROM books b
            LEFT JOIN ratings rt ON rt.book_id = b.book_id
            LEFT JOIN authors a ON a.author_id = b.author_id
            WHERE b.book_id IN (
                                SELECT b.book_id
                                FROM books AS b
                                LEFT JOIN ratings AS rt ON rt.book_id = b.book_id
                                GROUP BY b.book_id
                                HAVING COUNT(rating_id) >= 50
                                )
            GROUP BY author
            ORDER BY avg_rating DESC

            
      '''
select(sql)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.25
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,William Shakespeare/Paul Werstine/Barbara A. M...,3.79
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
9,Lois Lowry,3.75


Cамая высокая средняя оценка книг, учитывая только книги с 50 и более оценками, у соавторов книг о Гарри Поттере - писательницы Джоан Роулинг и иллюстратора Мэри Гранпрэ.

#### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [30]:
# выполним sql-запрос
sql = ''' 
            SELECT cast(AVG(reviews_count) AS numeric(4, 2)) avg_reviews_count
            FROM (
                  SELECT username,
                         COUNT(review_id) AS reviews_count
                  FROM reviews
                  WHERE username IN (
                                     SELECT username
                                     FROM ratings
                                     GROUP BY username
                                     HAVING COUNT(rating_id) > 50
                                     )
                  GROUP BY username
                  ) dt
      '''
select(sql)

,avg_reviews_count
0,24.33


Пользователи, которые поставили более 50 оценок, в среднем подготавливали 24,33 обзора.

## Общий вывод
<a name="conclusion"></a>

1.  Из 1000 книг, имеющихся в базе данных, после 1 января 2000 года была опубликована 821.      
2.  Максимальное количество обзоров пользователей у книги "Сумерки" писательницы Стефани Майер - 7, однако ее средняя пользовательская оценка достаточно невысока - всего 3,66. У 6 книг обзоры отсутствуют. Максимальная средняя пользовательская оценка - 5,0 - у 44 книг, при этом максимальное количество обзоров у них - 4. Минимальная средняя пользовательская оценка - 1,5 - у одной книги - "Забрать любовь" писательницы Джоди Пиколт.   
3.  Максимальное количество книг толще 50 страниц - 42 - выпустило издательство "Penguin Books".   
4.  Cамая высокая средняя оценка книг, учитывая только книги с 50 и более оценками, у соавторов книг о Гарри Поттере - писательницы Джоан Роулинг и иллюстратора Мэри Гранпрэ.   
5.  Пользователи, которые поставили более 50 оценок, в среднем подготавливали 24,33 обзора.